### **1. Parsing the System of Equations (1 point)**

In [15]:
import pathlib

def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    system = path.open()
    A = []
    B = []
    for equation in system:
        x,op1,y,op2,z,_,b = equation.split()
        B.append(float(b))
        
        x1 = x.replace('x', '')
        y = op1+y
        y1 = y.replace('y', '')
        z = op2+z
        z1 = z.replace('z', '')
        
        match x1:
            case '':
                x = 1.0
            case '-':
                x = -1.0
            case _:
                x = float(x1)
        match y1:
            case '+':
                y = 1.0
            case '-':
                y = -1.0
            case _:
                y = float(y1)
        match z1:
            case '+':
                z = 1.0
            case '-':
                z = -1.0
            case _:
                z = float(z1)
        A.append([x,y,z])
                
    return A,B    

A, B = load_system(pathlib.Path("equations_input.txt"))
print(f"{A=} {B=}")

A=[[2.0, 3.0, -1.0], [1.0, -1.0, 4.0], [3.0, 1.0, 2.0]] B=[5.0, 6.0, 7.0]


In [16]:
import re

def load_system_regex(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    file = path.open()
    A = []
    B = []
    pattern = r'([+-]?\d*)x\s*([+-]?\d*)y\s*([+-]?\d*)z\s*=\s*([+-]?\d+)'
    for equation in file:
        match = re.match(pattern, equation.replace(" ", ""))
        if match:
            if match.group(1) == '+' or match.group(1) == '':
                coefficient_x = 1
            elif match.group(1) == '-':
                coefficient_x = -1
            else:
                coefficient_x = float(match.group(1))

            if match.group(2) == '+' or match.group(2) == '':
                coefficient_y = 1
            elif match.group(2) == '-':
                coefficient_y = -1
            else:
                coefficient_y = float(match.group(2))
            
            if match.group(3) == '+' or match.group(3) == '':
                coefficient_z = 1
            elif match.group(3) == '-':
                coefficient_z = -1
            else:
                coefficient_z = float(match.group(3))

            if match.group(4) == '+' or match.group(4) == '':
                value = 1
            elif match.group(4) == '-':
                value = -1
            else:
                value = float(match.group(4))

            A.append([coefficient_x, coefficient_y, coefficient_z])
            B.append(value)
    return A, B

A, B = load_system_regex(pathlib.Path("equations_input.txt"))
print(f"{A=} {B=}")

A=[[2.0, 3.0, -1], [1, -1, 4.0], [3.0, 1, 2.0]] B=[5.0, 6.0, 7.0]


In [17]:
def load_system_1(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    i_index_A, j_index_A, index_B = 0, 0, 0
    A, B = [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [0, 0, 0]
    
    file = path.open()
    for line in file:
        for member in line.split():
            if member == '+':
                sign = False
            elif member == '-':
                sign = True
            elif len(member) > 1: #membrul este de tip cifra*necunoscuta
                if sign:
                    A[i_index_A][j_index_A] = float(member[0])
                    j_index_A = j_index_A + 1
                else: 
                    A[i_index_A][j_index_A] = -float(member[0])
                    j_index_A = j_index_A + 1
            elif len(member) <= 1: #membrul este de tip cifra sau necunoscuta
                if member not in {'x', 'y', 'z'} and member != '=': #membrul este cifra
                    B[index_B] = float(member)
                elif member != '=': #membrul este de tip semn necunoscuta
                    if sign :
                       A[i_index_A][j_index_A] = -1
                       j_index_A = j_index_A + 1
                    else:
                       A[i_index_A][j_index_A] = 1  
                       j_index_A = j_index_A + 1
        i_index_A, index_B = i_index_A + 1, index_B + 1
    return A, B

### **2. Matrix and Vector Operations (5 points)**

#### 2.1. Determinant

In [18]:
def determinant(matrix: list[list[float]]) -> float:
    if len(matrix) == 2 and len(matrix[0]) == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    elif len(matrix) == 3 and len(matrix[0]) == 3:
        return matrix[0][0] * (matrix[1][1] * matrix[2][2] - matrix[1][2] * matrix[2][1]) - matrix[0][1] * (matrix[1][0] * matrix[2][2] - matrix[1][2] * matrix[2][0]) + matrix[0][2] * (matrix[1][0] * matrix[2][1] - matrix[1][1] * matrix [2][0])

print(f"{determinant(A)=}")

determinant(A)=14.0


#### 2.2. Trace

In [19]:
def trace(matrix: list[list[float]]) -> float:
    return matrix[0][0] + matrix[1][1] + matrix[2][2]

print(f"{trace(A)=}")

trace(A)=3.0


#### 2.3. Vector norm

In [20]:
import math
def norm(vector: list[float]) -> float:
    return math.sqrt(vector[0] ** 2 + vector[1] ** 2 + vector[2] ** 2)

print(f"{norm(B)=}")

norm(B)=10.488088481701515


#### 2.4. Transpose of matrix

In [21]:
def transpose(matrix):

    transposed_matrix = [[0] * 3 for _ in range(3)] #[[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    
    for i in range(3):
        for j in range(3):
            transposed_matrix[j][i] = matrix[i][j]
    return transposed_matrix


print(f"{transpose(A)=}")

transpose(A)=[[2.0, 1, 3.0], [3.0, -1, 1], [-1, 4.0, 2.0]]


#### 2.5. Matrix-vector multiplication

In [22]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
    result = [0, 0, 0]
    
    for line in range(0, 3):
        for col in range(0, 3):
            result[line] += matrix[line][col] * vector[col] 
    return result

print(f"{multiply(A, B)=}")

multiply(A, B)=[21.0, 27.0, 35.0]


### **3. Solving using Cramer's Rule (1 point)**

In [23]:
def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:

    Ax = [[B[i]] + row[1:] for i, row in enumerate(matrix)]
    Ay = [[row[0]] + [B[i]] + [row[2]] for i, row in enumerate(matrix)]
    Az = [row[:2] + [B[i]] for i, row in enumerate(matrix)]
    return [determinant(Ax)/determinant(matrix), determinant(Ay)/determinant(matrix), determinant(Az)/determinant(matrix)]

print(f"{solve_cramer(A, B)=}")

solve_cramer(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


### **4. Solving using Inversion (3 points)**

In [24]:
def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
    return [
        [value for col_index, value in enumerate(row) if col_index != j]
        for row_index, row in enumerate(matrix) if row_index != i  
    ]

def cofactor(matrix: list[list[float]]) -> list[list[float]]:
    return [
        [((-1) ** (i+j)) * determinant(minor(matrix, i, j)) for j in range(3)]
        for i in range(3)
    ]

def adjoint(matrix: list[list[float]]) -> list[list[float]]:
    return transpose(cofactor(matrix))

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
    result = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    f = 1/determinant(matrix)
    adj = adjoint(matrix)
    for i in range(3):
        for j in range(3):
            result[i][j] = adj[i][j] * f
    return multiply(result, vector)

print(f"{solve(A, B)=}")

solve(A, B)=[0.35714285714285765, 2.071428571428571, 1.9285714285714293]
